In [34]:
import hashlib
import itertools
import json
import random
from time import sleep
from typing import List, Dict, Tuple

import requests
from tqdm import tqdm


In [35]:
RAW_DATA_DIR: str = './raw'

In [36]:
# experiment setup

integrations: List[dict] = [
    {
        "provider": "huggingFace",
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
    },
    #{
    #    "provider": "OpenAI",
    #    "model": "gpt-3.5-turbo"
    #}
]

personas: List[List[str]] = [
    ["neutral"],
    ["neutral", "progressive"],
    ["neutral", "polit-conservative"],
    ["neutral", "far-right"],
    ["neutral", "radical-left"],
    #["troll"],
    #["teenager, "internet-language", "troll", "radical-left"],
    #["teenager", "internet-language", "troll", "far-right"],
    # ["conservative", "polit-conservative"],
    # ["pensioner", "conservative"],
    ["dystopian"],
    ["male-college-student", "internet-language", "dystopian", "progressive"],
    ["carpenter", "dystopian", "polit-conservative"],
    ["devilsadvocate", "far-right"],
    ["male-college-student", "internet-language", "devilsadvocate", "radical-left"],
    ["pensioner", "devilsadvocate", "far-right"],
    ["know-it-all", "progressive"],
    ["hustler", "know-it-all", "far-right"],
    ["male-college-student", "internet-language", "know-it-all", "radical-left"],
    ["phd-student", "mediator", "progressive"],
    ["affluent-young-woman", "mediator", "polit-conservative"],
    ["educator"],
    ["privileged-woman", "karen", "far-right"],
    ["affluent-young-woman", "karen", "polit-conservative"],
    ["misogynist"],
    ["teenager", "internet-language", "misogynist"],
    ["carpenter", "misogynist", "far-right"],
    ["esoteric"],
    ["privileged-woman", "esoteric", "far-right"],
    ["hippie"],
    ["privileged-woman", "hippie", "radical-left"],
    ["phd-student", "supportive-saviour", "progressive"],
    ["hustler", "influencer", "progressive"],
    ["wealthy-entrepreneur", "influencer", "polit-conservative"],
    ["affluent-young-woman", "social-media-activist", "radical-left"],
    ["male-college-student", "internet-language", "social-media-activist", "far-right"]
]
#languages: List[str] = ["Dutch", "English", "German"]
#platforms: List[str] = ["Twitter", "Reddit"] 
RAW_POSTS_DIR: str = "./sample_og_posts.json"
threads: List[Dict[str, str]] = json.load(open(RAW_POSTS_DIR, 'r'))  

In [37]:
# number of iterations
x: int = 1
# hidden parameters (randomly chosen)
length: List[str] = ["50", "100", "200", "280"] 

In [38]:
len(threads)

15

In [39]:
configurations: List[Tuple] = list(
    itertools.product(*[integrations, personas, threads])
)
random.shuffle(configurations)
len(configurations)

465

In [40]:
configurations[0]

({'provider': 'huggingFace', 'model': 'mistralai/Mistral-7B-Instruct-v0.2'},
 ['male-college-student', 'internet-language', 'dystopian', 'progressive'],
 {'persona': 'liberal',
  'model': 'gpt-3.5-turbo',
  'topic': 'Healthcare',
  'platform': 'Twitter',
  'language': 'English',
  'text': "Access to quality healthcare should be a fundamental right, not a privilege. We need a system that ensures everyone can receive the care they need, regardless of income or social status. Let's prioritize public options and work towards a healthier, more equitable society.",
  'annotation': {'topic': None, 'persona': None, 'authenticity': None},
  'id': '93cc14194d55075d6662b345e343b34b6d51f2d1d46b0871'})

In [ ]:
for _ in range(x):
    for integration, persona, thread in tqdm(configurations):

        payload: dict = {
            "personas": persona,
            "integration": integration,
            "thread": {'posts': [{'author': thread['persona'], 'message': thread['text']}]},
            "length": random.choice(length)
        }

        response = requests.post('http://127.0.0.1:8000/reply/', json=payload)

        if response.status_code == 500 or response.status_code == 502:
            print("500/502: Connection Error, too many request, try again later.")
            break

        try:
            data: dict = response.json()

        except Exception as e:
            print(e, ':', response)
            break
        
        sample: dict = {
            "persona": data['persona']['id'],
            "model": integration["model"],
            "threads": thread['text'],
            "topic": thread["topic"],
            "text": data['response'],
            "payload": data,
            "annotation": {
                "politics": None,
                "persona": None,
                "tone": None,
                "authenticity": None
            }
        }
        print(data['response'])
        sample['id'] = hashlib.shake_256(str.encode(json.dumps(sample))).hexdigest(24)
        open(f'{RAW_DATA_DIR}/{sample["id"]}.json', 'w').write(json.dumps(sample, indent=4))
        sleep(1)







  0%|          | 0/465 [00:00<?, ?it/s]

@young_and_angry: agreed, man. but how r we gonna pay for it all? capitalism sucks ass. we need a real revolution, not just more band-aids.







  0%|          | 1/465 [00:02<22:57,  2.97s/it]

@anarchist_hippie: 💩💩💩 recycling is for the weak. real change comes from disrupting the system, not conforming to it. 💣💣💣







  0%|          | 2/465 [00:05<22:54,  2.97s/it]

...

💡💻 agree. invest in edu tech & green jobs. let's innovate & create opportunities. 💪🏽🤝 solidarity for all.







 11%|█▏        | 53/465 [03:40<20:24,  2.97s/it]

@alt_right, capitalist pigs hoarding wealth while the masses suffer. war profiteers rake in billions. communism for the people! smash the system!







 12%|█▏        | 54/465 [03:42<19:00,  2.77s/it]

damn straight, but where's the line? gov't overreach is a slippery slope. freedom's important, but so is public safety. balance is key. but how long till they push it too far?
